In [1]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

import itertools

def scs(ss):
    """ Returns shortest common superstring of given
        strings, which must be the same length """
    shortest_sup = None
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if shortest_sup is None or len(sup) < len(shortest_sup):
            shortest_sup = sup  # found shorter superstring
    return shortest_sup  # return shortest

In [6]:
shortest_common_superstring=scs([ 'ABC','BCA','CAB'])
print("The shortest common superstrings from the set of strings {'ABC','BCA','CAB'} is", shortest_common_superstring)

The shortest common superstrings from the set of strings {'ABC','BCA','CAB'} is ABCAB


In [10]:
length_SCS=len(scs(['CCT','TGC','TGG','GAT','ATT']))
print("The length of the shortest common superstrings from the set of strings {'CCT','TGC','TGG','GAT','ATT'} is", length_SCS)

Thelength of the shortest common superstrings from the set of strings {'CCT','TGC','TGG','GAT','ATT'} is 10


In [4]:
import itertools

def scs_list(ss):
    """ Returns a list of all shortest common superstring of given
        strings, which must be the same length """
    shortest_sup = []
    shortest_len = None
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if shortest_len is None or len(sup) < shortest_len:
            shortest_len = len(sup)
            shortest_sup = [sup]  # reset to the new shortest superstring
        elif len(sup) == shortest_len:
            shortest_sup.append(sup)
    return shortest_sup  # return shortest

In [11]:
# Returns just one shortest superstring
strings = ['ABC', 'BCA', 'CAB']
shortest_common_superstring=scs(strings)
list_scs=scs_list(strings)
print("The shortest common superstrings from the set of strings {'ABC','BCA','CAB'} using scs algorithm:", shortest_common_superstring)
print("The shortest common superstrings from the set of strings {'ABC','BCA','CAB'} using scs_list algorithm:", list_scs)

The shortest common superstrings from the set of strings {'ABC','BCA','CAB'} using scs algorithm: ABCAB
The shortest common superstrings from the set of strings {'ABC','BCA','CAB'} using scs_list algorithm: ['ABCAB', 'BCABC', 'CABCA']


In [17]:
!curl -o ads1_week4_reads.fq https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ads1_week4_reads.fq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  386k  100  386k    0     0  1770k      0 --:--:-- --:--:-- --:--:-- 1831k


In [19]:
def readFastq(filename):
    """Reads the fasta file"""
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities
fastq,qual = readFastq('ads1_week4_reads.fq')

def overlap_all_pairs(reads, k):
    """Makes a kmer dictionary of all the reads and then checks for only reads that have a match in the kmer index and retures the overlaps"""
    kmer_dict = {}
    for read in reads:
        for i in range(len(read) - k + 1):
            kmer = read[i:i+k]
            if kmer not in kmer_dict:
                kmer_dict[kmer] = set()
            kmer_dict[kmer].add(read)
    
    olaps = []
    for read in reads:
        a = read[-k:]
        if a in kmer_dict:
            b_set = kmer_dict[a]
            for b in b_set:
                if b != read:
                    olen = overlap(read, b, k)
                    if olen > 0:
                        olaps.append((read, b))
    return olaps

def greedy_assemble(reads, k):
    """Greedy scs algorith using kmer dictionary"""
    reads = set(reads)
    while True:
        olaps = overlap_all_pairs(reads, k)

        if not olaps:
            break

        max_olen = 0
        best_pair = None
        for a, b in olaps:
            olen = overlap(a, b, min_length=k)
            if olen > max_olen:
                max_olen = olen
                best_pair = (a, b)

        if best_pair is None:
            break

        a, b = best_pair
        reads.remove(a)
        reads.remove(b)
        reads.add(a + b[max_olen:])

    return max(reads, key=len)

In [20]:
fastq, qual = readFastq('ads1_week4_reads.fq')
assembled_genome = greedy_assemble(fastq, k=30)

print(f"Assembled genome length: {len(assembled_genome)}")
print(f"Number of A's: {assembled_genome.count('A')}")
print(f"Number of T's: {assembled_genome.count('T')}")

Assembled genome length: 15894
Number of A's: 4633
Number of T's: 3723
